In [1]:
import pandas as pd

In [10]:
data_raw = pd.read_csv("housing/data_raw/tabula_GB_complete.csv")

In [14]:
Bcodes = data_raw.Code_BuildingVariant 

In [24]:
import importlib
import sys
from datetime import datetime
from hisim import log
import hisim.simulator as sim
import os
#from hisim.postprocessing import postprocessing_main as pp

def main(path_to_module: str, function_in_module: str, index ,my_simulation_parameters = None): # Add a new parameter: index
    log.information("#################################")
    log.information("starting simulation of " + path_to_module  + " " + function_in_module)
    starttime = datetime.now()
    d4 = starttime.strftime("%d-%b-%Y %H:%M:%S")
    log.information("Start @ " + d4 + " "  )
    log.profile(path_to_module  + " " + function_in_module + "Start @ " + d4 )
    log.information("#################################")
    normalized_path = os.path.normpath(path_to_module)
    path_in_list = normalized_path.split(os.sep)
    module_filename = path_in_list[-1]
    if len(path_in_list) >= 1:
        path_to_be_added = os.path.join(os.getcwd(), *path_in_list[:-1])
        if os.path.isdir(path_to_be_added):
            # Add current path to PYTHONPATH
            sys.path.append(path_to_be_added)
            #for dirs in os.walk(path_to_be_added):
            #    sys.path.append(dirs)
        else:
            raise ValueError("Directory location of module location is nonexistent!\nDirectory entered: {}".format(path_to_be_added))
    suffix =module_filename[-3:]
    if suffix != ".py":
        module_fullfilename = "{}.py".format(module_filename)
    else:
        module_fullfilename = module_filename
        module_filename = module_filename[:-3]
    filepath = os.path.join(path_to_be_added, module_fullfilename)
    if os.path.isfile(filepath):
        # Get setup function to executable
        targetmodule = importlib.import_module(module_filename)
    else:
        raise ValueError("Python script {}.py could not be found".format(module_filename))

    # Create a Simulator object based on setup function
    my_sim: sim.Simulator = sim.Simulator(module_directory=path_to_be_added,
                                          setup_function=function_in_module,
                                          index=index, # Add a new parameter: index
                                          my_simulation_parameters=my_simulation_parameters) 

    # Build method
    model_init_method = getattr(targetmodule, function_in_module)

    # Pass setup function to simulator
    model_init_method(my_sim, my_simulation_parameters)

    # Perform simulation throughout the defined timeline
    my_sim.run_all_timesteps()
    log.information("#################################")
    endtime = datetime.now()
    d4 = endtime.strftime("%d-%b-%Y %H:%M:%S")
    log.information("finished @ " + d4)
    log.profile("finished @ " + d4)
    log.profile("duration: " + str((endtime-starttime).total_seconds()))
    log.information("#################################")
    log.information("")


In [ ]:

def basic_household_explicit(my_sim, my_simulation_parameters: Optional[SimulationParameters] = None):
    """
    This setup function emulates an household including
    the basic components. Here the residents have their
    electricity and heating needs covered by the photovoltaic
    system and the heat pump.

    - Simulation Parameters
    - Components
        - Occupancy (Residents' Demands)
        - Weather
        - Photovoltaic System
        - Building
        - Heat Pump
    """
    intensity= carbon_data_scotland() 
    print(intensity)
    y=intensity['data'][0]
    print(y)
    
    #forecast= intensity['data'][0]['data'][0]('intensity').get('forecast')
    #x=json.load(intensity)
    #print(x)
    #print(forecast)

    ##### System Parameters #####

    # Set simulation parameters
    year = 2021
    seconds_per_timestep = 60

    # Set weather
    location = "Aachen"

    # Set photovoltaic system
    time = 2019
    power = 10E3
    load_module_data = False
    module_name = "Hanwha_HSL60P6_PA_4_250T__2013_"
    integrateInverter = True
    inverter_name = "ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_"

    # Set occupancy
    occupancy_profile = "CH01"

    # Set building
    building_code = "DE.N.SFH.05.Gen.ReEx.001.002"
    building_class = "medium"
    initial_temperature = 23

    # Set heat pump controller
    t_air_heating = 16.0
    t_air_cooling = 24.0
    offset = 0.5
    hp_mode = 2

    # Set heat pump
    hp_manufacturer = "Viessmann Werke GmbH & Co KG"
    hp_name = "Vitocal 300-A AWO-AC 301.B07"
    hp_min_operation_time = 60
    hp_min_idle_time = 15

    ##### Build Components #####

    # Build system parameters
    if my_simulation_parameters is None:
        my_simulation_parameters = SimulationParameters.full_year_all_options(year=year,
                                                                                 seconds_per_timestep=seconds_per_timestep)
    my_sim.SimulationParameters = my_simulation_parameters
    # Build occupancy
    my_occupancy = occupancy.Occupancy(profile_name=occupancy_profile, my_simulation_parameters=my_simulation_parameters)
    my_sim.add_component(my_occupancy)

    # Build Weather
    my_weather = weather.Weather(location=location, my_simulation_parameters= my_simulation_parameters)
    my_sim.add_component(my_weather)

    my_photovoltaic_system = pvs.PVSystem(time=time,
                                          location=location,
                                          power=power,
                                          load_module_data=load_module_data,
                                          module_name=module_name,
                                          integrateInverter=integrateInverter,
                                          inverter_name=inverter_name,
                                          my_simulation_parameters=my_simulation_parameters)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.TemperatureOutside,
                                         my_weather.ComponentName,
                                         my_weather.TemperatureOutside)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.DirectNormalIrradiance,
                                         my_weather.ComponentName,
                                         my_weather.DirectNormalIrradiance)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.DirectNormalIrradianceExtra,
                                         my_weather.ComponentName,
                                         my_weather.DirectNormalIrradianceExtra)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.DiffuseHorizontalIrradiance,
                                         my_weather.ComponentName,
                                         my_weather.DiffuseHorizontalIrradiance)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.GlobalHorizontalIrradiance,
                                         my_weather.ComponentName,
                                         my_weather.GlobalHorizontalIrradiance)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.Azimuth,
                                         my_weather.ComponentName,
                                         my_weather.Azimuth)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.ApparentZenith,
                                         my_weather.ComponentName,
                                         my_weather.ApparentZenith)
    my_photovoltaic_system.connect_input(my_photovoltaic_system.WindSpeed,
                                         my_weather.ComponentName,
                                         my_weather.WindSpeed)
    my_sim.add_component(my_photovoltaic_system)


    my_base_electricity_load_profile = sumbuilder.ElectricityGrid(name="BaseLoad",
                                                                      grid=[my_occupancy, "Subtract", my_photovoltaic_system ], my_simulation_parameters=my_simulation_parameters)
    my_sim.add_component(my_base_electricity_load_profile)

    my_building = building.Building(building_code=building_code,
                                        bClass=building_class,
                                        initial_temperature=initial_temperature,
                                        my_simulation_parameters=my_simulation_parameters)
    my_building.connect_input(my_building.Altitude,
                              my_weather.ComponentName,
                              my_building.Altitude)
    my_building.connect_input(my_building.Azimuth,
                              my_weather.ComponentName,
                              my_building.Azimuth)
    my_building.connect_input(my_building.DirectNormalIrradiance,
                              my_weather.ComponentName,
                              my_building.DirectNormalIrradiance)
    my_building.connect_input(my_building.DiffuseHorizontalIrradiance,
                              my_weather.ComponentName,
                              my_building.DiffuseHorizontalIrradiance)
    my_building.connect_input(my_building.GlobalHorizontalIrradiance,
                              my_weather.ComponentName,
                              my_building.GlobalHorizontalIrradiance)
    my_building.connect_input(my_building.DirectNormalIrradianceExtra,
                              my_weather.ComponentName,
                              my_building.DirectNormalIrradianceExtra)
    my_building.connect_input(my_building.ApparentZenith,
                             my_weather.ComponentName,
                             my_building.ApparentZenith)
    my_building.connect_input(my_building.TemperatureOutside,
                              my_weather.ComponentName,
                              my_weather.TemperatureOutside)
    my_building.connect_input(my_building.HeatingByResidents,
                              my_occupancy.ComponentName,
                              my_occupancy.HeatingByResidents)
    my_sim.add_component(my_building)

    my_heat_pump_controller = heat_pump.HeatPumpController(t_air_heating=t_air_heating,
                                                           t_air_cooling=t_air_cooling,
                                                           offset=offset,
                                                           mode=hp_mode,
                                                           my_simulation_parameters=my_simulation_parameters)
    my_heat_pump_controller.connect_input(my_heat_pump_controller.TemperatureMean,
                                          my_building.ComponentName,
                                          my_building.TemperatureMean)
    my_heat_pump_controller.connect_input(my_heat_pump_controller.ElectricityInput,
                                          my_base_electricity_load_profile.ComponentName,
                                          my_base_electricity_load_profile.ElectricityOutput)
    my_sim.add_component(my_heat_pump_controller)

    my_heat_pump = heat_pump.HeatPump(manufacturer=hp_manufacturer,
                                          name=hp_name,
                                          min_operation_time=hp_min_operation_time,
                                          min_idle_time=hp_min_idle_time,
                                      my_simulation_parameters=my_simulation_parameters)
    my_heat_pump.connect_input(my_heat_pump.State,
                               my_heat_pump_controller.ComponentName,
                               my_heat_pump_controller.State)
    my_heat_pump.connect_input(my_heat_pump.TemperatureOutside,
                               my_weather.ComponentName,
                               my_weather.TemperatureOutside)

    my_sim.add_component(my_heat_pump)

    my_building.connect_input(my_building.ThermalEnergyDelivered,
                              my_heat_pump.ComponentName,
                              my_heat_pump.ThermalEnergyDelivered)

